<a href="https://colab.research.google.com/github/scwandy/Crypto/blob/main/Crypto_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the full token list from Coingekco

In [30]:
import requests
import csv

url = 'https://api.coingecko.com/api/v3/coins/list'
response = requests.get(url)

coins = response.json()

with open('coin_list.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'symbol', 'name'])
    for coin in coins:
        writer.writerow([coin['id'], coin['symbol'], coin['name']])

In [31]:
coin_list.describe()

,id,symbol,name
count,10702,10701,10702
unique,10702,9052,10702
top,01coin,gold,01coin
freq,1,8,1


In [32]:
import pandas as pd

coin_list = pd.read_csv('coin_list.csv')
print(coin_list.head())

                         id symbol                       name
0                    01coin    zoc                     01coin
1                    0chain    zcn                        Zus
2             0vix-protocol    vix              0VIX Protocol
3                        0x    zrx                0x Protocol
4  0x0-ai-ai-smart-contract    0x0  0x0.ai: AI Smart Contract


In [34]:
# Get latest coin data from CoinGecko API
url = 'https://api.coingecko.com/api/v3/coins/markets'
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': '100',
    'page': '1',
    'sparkline': 'false'
}
response = requests.get(url, params=params)
data = response.json()

# Convert JSON data to pandas DataFrame
df = pd.json_normalize(data)

# Calculate last close volume
df['last_close_volume'] = df['market_cap'] - df['market_cap_change_24h']

# Calculate 15-minute average of volume
df['turnover_15min_avg'] = df['total_volume'] \
    .rolling(window=20, min_periods=15) \
    .mean() \
    .shift(1) \
    .fillna(df['total_volume'])

# Identify coins where last closing volume is 5 times bigger than 15-minute average
target_list = df[df['last_close_volume'] > 5 * df['turnover_15min_avg']]['symbol'].tolist()

print(target_list)

['btc', 'eth', 'usdt', 'bnb', 'usdc', 'xrp', 'ada', 'steth', 'doge', 'matic', 'sol', 'dot', 'ltc', 'busd', 'trx', 'ton', 'okb', 'xmr', 'etc', 'icp', 'xlm', 'fil', 'bch', 'tusd', 'hbar', 'apt', 'cro', 'ldo', 'arb', 'near', 'qnt', 'vet', 'ape', 'algo', 'grt', 'ftm', 'eos', 'sand', 'xrd', 'egld', 'usdp', 'aave', 'theta', 'mana', 'stx', 'frax', 'edgt', 'imx', 'xtz', 'flow', 'axs', 'rndr', 'rpl', 'kcs', 'snx', 'neo', 'crv', 'gt', 'bit', 'usdd', 'klay', 'op', 'wbt', 'chz', 'cfx', 'lunc', 'bsv', 'cspr', 'gmx', 'inj', 'mkr', 'mina', 'btt', 'xec', 'ht', 'dash', 'reth', 'fxs', 'miota', 'xdc', 'bgb', 'cake', 'tkx', 'paxg', 'ceth', 'twt', 'xaut', 'kas', 'gusd', 'zil', 'flr', 'osmo']
